In [1]:
from itertools import islice

a = map(lambda x: x - 1, range(10))
slice_of_a = islice(a, 5)
slice_of_a

In [2]:
list(slice_of_a)

[-1, 0, 1, 2, 3]

In [3]:
import polars as pl

df = pl.DataFrame({"a": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]})
df

a
i64
1
2
3
4
5
6
7
8
9


In [4]:
df2 = pl.DataFrame({"a": [None, None, None, None, None]}, schema={"a": pl.Null})
df2

a
null
null
null
null
null
null


In [5]:
df3 = pl.concat([df2, df], how="diagonal_relaxed")
df3

a
i64
null
null
null
null
null
1
2
3
4


In [6]:
def get_join_table(
    left: pl.LazyFrame,
    right: pl.LazyFrame,
    left_on: set[str],
    right_pkey: str,
) -> pl.LazyFrame:
    left_on = left_on.intersection(left.columns)
    left_join_table = left.select(left_on)
    joins = map(
        lambda x: left_join_table.join(
            right, left_on=x, right_on=right_pkey, how="inner"
        ),
        left_on,
    )

    left_join_table = pl.concat(joins)

    return left_join_table

In [7]:
df1 = pl.LazyFrame(
    {
        "a": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
        "aa": [11, 22, 33, 4, 5, 6, 7, 8, 9, 10, None],
        "borrow": [
            "38393939",
            "43243434",
            "barto",
            "beano",
            "super",
            "blyat",
            "blyat",
            "blyat",
            "blyat",
            "blyat",
            "blyat",
        ],
    }
)

df2 = pl.LazyFrame(
    {
        "bean": [1, 1, 2, 2, 3, 3, 4, 5, 6, 7, 8, 9, 10, 11, 22],
        "b": [
            "cat",
            "dog",
            "bean",
            "soup",
            "cat",
            "dog",
            "bean",
            "soup",
            "cat",
            "dog",
            "bean",
            "soup",
            "cat",
            "eeeeee",
            "000000",
        ],
    },
)

# df2 = df2.group_by("bean").agg("b")

df3 = df1.join(df2, left_on="a", right_on="bean", how="left")
df3 = df3.join(df2, left_on="aa", right_on="bean", how="left")
df3.collect()

a,aa,borrow,b,b_right
i64,i64,str,str,str
1,11,"""38393939""","""cat""","""eeeeee"""
1,11,"""38393939""","""dog""","""eeeeee"""
2,22,"""43243434""","""bean""","""000000"""
2,22,"""43243434""","""soup""","""000000"""
3,33,"""barto""","""cat""",null
3,33,"""barto""","""dog""",null
4,4,"""beano""","""bean""","""bean"""
5,5,"""super""","""soup""","""soup"""
6,6,"""blyat""","""cat""","""cat"""


In [8]:
join_table = get_join_table(df1, df2, {"a", "aa"}, "bean")
join_table.collect()

aa,a,b
i64,i64,str
4,4,"""bean"""
5,5,"""soup"""
6,6,"""cat"""
7,7,"""dog"""
8,8,"""bean"""
9,9,"""soup"""
10,10,"""cat"""
11,1,"""eeeeee"""
22,2,"""000000"""


In [9]:
join_table = join_table.group_by("a").agg("b")
df1 = df1.join(join_table, on="a", how="left")
df1.collect()

a,aa,borrow,b
i64,i64,str,list[str]
1,11,"""38393939""","[""eeeeee"", ""cat"", ""dog""]"
2,22,"""43243434""","[""000000"", ""bean"", ""soup""]"
3,33,"""barto""","[""cat"", ""dog""]"
4,4,"""beano""","[""bean"", ""bean""]"
5,5,"""super""","[""soup"", ""soup""]"
6,6,"""blyat""","[""cat"", ""cat""]"
7,7,"""blyat""","[""dog"", ""dog""]"
8,8,"""blyat""","[""bean"", ""bean""]"
9,9,"""blyat""","[""soup"", ""soup""]"


In [10]:
from typing import NamedTuple


class BallSack(NamedTuple):
    balls: int
    smoothness: float

    def spicyness(self):
        return self.balls * self.smoothness * temp


ballsack = BallSack(smoothness=0.9, balls=2)
temp = 69.420

In [11]:
ballsack.spicyness()

124.956

In [12]:
BallSack.spicyness(ballsack)

124.956

In [13]:
ballsacks = [BallSack(2, 0.9), BallSack(14, 0.7), BallSack(3, 0.69)]

In [14]:
spicynesses = map(BallSack.spicyness, ballsacks)
list(spicynesses)

[124.956, 680.3159999999999, 143.6994]

In [15]:
spicynesses = map(lambda x: x.spicyness(), ballsacks)
list(spicynesses)

[124.956, 680.3159999999999, 143.6994]

In [16]:
from pydantic import BaseModel


class Bingo(BaseModel):
    wins: int
    losses: int
    happyness: float
    chart_name: str


class Bongo(BaseModel):
    owner_name: str
    owner_id: int
    weight: float
    length: float
    width: float
    depth: float


class MyJsonData(BaseModel):
    bingos: list[Bingo]
    bongos: list[Bongo]

In [17]:
with open("my_json_data.json") as file:
    file_str = file.read()

data = MyJsonData.model_validate_json(file_str)
data

MyJsonData(bingos=[Bingo(wins=12, losses=9, happyness=0.8, chart_name='super_bingo'), Bingo(wins=34, losses=93, happyness=0.039, chart_name='loser_bingo'), Bingo(wins=123, losses=0, happyness=1.0, chart_name='cheater_bingo'), Bingo(wins=1343, losses=99393, happyness=0.3, chart_name='addict_bingo')], bongos=[Bongo(owner_name='isaac', owner_id=837483, weight=3948.0, length=384.0, width=3483.34, depth=9893.38), Bongo(owner_name='joe', owner_id=3748743, weight=34.0, length=384.234, width=3483.0, depth=8377.38)])

In [18]:
data.bingos

[Bingo(wins=12, losses=9, happyness=0.8, chart_name='super_bingo'),
 Bingo(wins=34, losses=93, happyness=0.039, chart_name='loser_bingo'),
 Bingo(wins=123, losses=0, happyness=1.0, chart_name='cheater_bingo'),
 Bingo(wins=1343, losses=99393, happyness=0.3, chart_name='addict_bingo')]

In [19]:
data.bongos

[Bongo(owner_name='isaac', owner_id=837483, weight=3948.0, length=384.0, width=3483.34, depth=9893.38),
 Bongo(owner_name='joe', owner_id=3748743, weight=34.0, length=384.234, width=3483.0, depth=8377.38)]

In [20]:
import json

json_obj = json.loads(file_str)
json_obj

{'bingos': [{'id': 6969,
   'wins': 12,
   'losses': 9,
   'happyness': 0.8,
   'chart_name': 'super_bingo'},
  {'id': 69420,
   'wins': 34,
   'losses': 93,
   'happyness': 0.039,
   'chart_name': 'loser_bingo'},
  {'id': 1234,
   'wins': 123,
   'losses': 0,
   'happyness': 1,
   'chart_name': 'cheater_bingo'},
  {'id': 420420,
   'wins': 1343,
   'losses': 99393,
   'happyness': 0.3,
   'chart_name': 'addict_bingo'}],
 'bongos': [{'bingo_ids': [6969, 420420],
   'owner_name': 'isaac',
   'owner_id': 837483,
   'weight': 3948,
   'length': 384,
   'width': 3483.34,
   'depth': 9893.38},
  {'bingo_ids': [1234, 69420],
   'owner_name': 'joe',
   'owner_id': 3748743,
   'weight': 34,
   'length': 384.234,
   'width': 3483,
   'depth': 8377.38}]}

In [21]:
num_games_for_each = map(lambda x: x["losses"] + x["wins"], json_obj["bingos"])
sum_games = sum(num_games_for_each)
sum_games

101007

In [22]:
num_games_for_each = map(lambda x: x.losses + x.wins, data.bingos)
sum_games = sum(num_games_for_each)
sum_games

101007

In [23]:
bingos = pl.LazyFrame(json_obj["bingos"])
bingos.collect()

id,wins,losses,happyness,chart_name
i64,i64,i64,f64,str
6969,12,9,0.8,"""super_bingo"""
69420,34,93,0.039,"""loser_bingo"""
1234,123,0,1.0,"""cheater_bingo"""
420420,1343,99393,0.3,"""addict_bingo"""


In [24]:
bongos = pl.LazyFrame(json_obj["bongos"])
bongos.collect()

bingo_ids,owner_name,owner_id,weight,length,width,depth
list[i64],str,i64,i64,f64,f64,f64
"[6969, 420420]","""isaac""",837483,3948,384.0,3483.34,9893.38
"[1234, 69420]","""joe""",3748743,34,384.234,3483.0,8377.38


In [25]:
bongos_join_table = bongos.select("bingo_ids", "owner_id").explode("bingo_ids")
bongos_join_table.collect()

bingo_ids,owner_id
i64,i64
6969,837483
420420,837483
1234,3748743
69420,3748743


In [26]:
bingos_to_bongos = bingos.join(bongos_join_table, left_on="id", right_on="bingo_ids")
bingos_to_bongos.collect()

id,wins,losses,happyness,chart_name,owner_id
i64,i64,i64,f64,str,i64
6969,12,9,0.8,"""super_bingo""",837483
420420,1343,99393,0.3,"""addict_bingo""",837483
1234,123,0,1.0,"""cheater_bingo""",3748743
69420,34,93,0.039,"""loser_bingo""",3748743


In [27]:
bingos_to_bongos = bingos_to_bongos.group_by("owner_id").agg(
    pl.struct(pl.all().exclude("owner_id")).alias("bingos")
)
bingos_to_bongos.collect()

owner_id,bingos
i64,list[struct[5]]
837483,"[{6969,12,9,0.8,""super_bingo""}, {420420,1343,99393,0.3,""addict_bingo""}]"
3748743,"[{1234,123,0,1.0,""cheater_bingo""}, {69420,34,93,0.039,""loser_bingo""}]"


In [28]:
bingos_to_bongos = bingos_to_bongos.join(bongos, on="owner_id")
bingos_to_bongos.collect()

owner_id,bingos,bingo_ids,owner_name,weight,length,width,depth
i64,list[struct[5]],list[i64],str,i64,f64,f64,f64
837483,"[{6969,12,9,0.8,""super_bingo""}, {420420,1343,99393,0.3,""addict_bingo""}]","[6969, 420420]","""isaac""",3948,384.0,3483.34,9893.38
3748743,"[{1234,123,0,1.0,""cheater_bingo""}, {69420,34,93,0.039,""loser_bingo""}]","[1234, 69420]","""joe""",34,384.234,3483.0,8377.38


In [29]:
bingos_to_bongos = bingos_to_bongos.drop("bingo_ids")
bingos_to_bongos.collect()

owner_id,bingos,owner_name,weight,length,width,depth
i64,list[struct[5]],str,i64,f64,f64,f64
837483,"[{6969,12,9,0.8,""super_bingo""}, {420420,1343,99393,0.3,""addict_bingo""}]","""isaac""",3948,384.0,3483.34,9893.38
3748743,"[{1234,123,0,1.0,""cheater_bingo""}, {69420,34,93,0.039,""loser_bingo""}]","""joe""",34,384.234,3483.0,8377.38


In [31]:
bingos_to_bongos.collect().write_json(
    "bingos_to_bongos.json", row_oriented=True, pretty=True
)

In [32]:
bingos_to_bongos_json_obj = bingos_to_bongos.collect().to_dicts()
bingos_to_bongos_json_obj

[{'owner_id': 837483,
  'bingos': [{'id': 6969,
    'wins': 12,
    'losses': 9,
    'happyness': 0.8,
    'chart_name': 'super_bingo'},
   {'id': 420420,
    'wins': 1343,
    'losses': 99393,
    'happyness': 0.3,
    'chart_name': 'addict_bingo'}],
  'owner_name': 'isaac',
  'weight': 3948,
  'length': 384.0,
  'width': 3483.34,
  'depth': 9893.38},
 {'owner_id': 3748743,
  'bingos': [{'id': 1234,
    'wins': 123,
    'losses': 0,
    'happyness': 1.0,
    'chart_name': 'cheater_bingo'},
   {'id': 69420,
    'wins': 34,
    'losses': 93,
    'happyness': 0.039,
    'chart_name': 'loser_bingo'}],
  'owner_name': 'joe',
  'weight': 34,
  'length': 384.234,
  'width': 3483.0,
  'depth': 8377.38}]

In [33]:
MyJsonData.model_validate(json_obj)

MyJsonData(bingos=[Bingo(wins=12, losses=9, happyness=0.8, chart_name='super_bingo'), Bingo(wins=34, losses=93, happyness=0.039, chart_name='loser_bingo'), Bingo(wins=123, losses=0, happyness=1.0, chart_name='cheater_bingo'), Bingo(wins=1343, losses=99393, happyness=0.3, chart_name='addict_bingo')], bongos=[Bongo(owner_name='isaac', owner_id=837483, weight=3948.0, length=384.0, width=3483.34, depth=9893.38), Bongo(owner_name='joe', owner_id=3748743, weight=34.0, length=384.234, width=3483.0, depth=8377.38)])

In [1]:
arr = ["bogos", "bingos", "bongies", "logos", "chungus"]
for i, element in enumerate(arr):
    print(i, element)

0 bogos
1 bingos
2 bongies
3 logos
4 chungus


In [2]:
for i in range(len(arr)):
    print(i, arr[i])

0 bogos
1 bingos
2 bongies
3 logos
4 chungus


In [5]:
stran = "bingi"
reversed_strang = reversed(stran)
for char in enumerate(reversed_strang):
    print(char)

(0, 'i')
(1, 'g')
(2, 'n')
(3, 'i')
(4, 'b')


In [6]:
def words_without(words: list[str], target: str) -> list[str]:
    return list(filter(lambda x: x != target, words))


words = ["bogos", "bingos", "bongies", "logos", "chungus"]
words_without(words, "bingos")

['bogos', 'bongies', 'logos', 'chungus']

In [2]:
mask = {2, 3, 34, 12, 52}


def has_23(nums: list[int]) -> bool:
    num_in_mask = map(mask.__contains__, nums)
    return any(num_in_mask)


has_23([1, 6, 9, 39])

False